In [1]:
from __future__ import division
from __future__ import print_function  
import warnings
import pandas as pd
import gsp_support as gsp
import matplotlib.pyplot as plt
import numpy as np
pd.options.plotting.backend = "plotly"
import datetime

In [ ]:
#######################################################################################################################

In [ ]:
# read the gsp data




In [ ]:
#######################################################################################################################

In [ ]:
# filtering stage 1

for meter in ac_list:
    
    
    # write the propoer method of reading the input and the meter wise dataframe in gsp_result #TODO
    
   





    # # plot(df, gsp_result, meter,0)
    # gsp_result.to_csv(folder+'gsp_'+meter+'.csv')

    # # check 1 - to be converted in a function


    gsp_result['Power'] = e
    gsp_result['Current'] = df['z_Avg Current (Amp)']
    gsp_result['Comp'] = gsp_result[0].apply(lambda x: 1 if x > 0 else 0)


    #inits

    all_ = []
    powercut = []



    # init = {'Ton':np.nan,'Toff':np.nan}
    d = {'Ton':np.nan,'Toff':np.nan}
    p = {'Ton_':np.nan,'Toff_':np.nan}

    # print(init)
    # print(d)
    # print(p)

    remove = []
    pp = 0
    # iterate on the daraframe det_comp

    ind = gsp_result.shape[0]-1
    for i in range(0,ind):
        if gsp_result.iloc[i+1]['Comp'] > gsp_result.iloc[i]['Comp']:

            d['Ton'] = gsp_result.iloc[i+1].name
            if gsp_result.iloc[i]['Power']==0:
    #             print(gsp_result.iloc[i])
    #             print('-----------------------')
                p['Ton_'] = d['Ton']
    #             print(pp)
    #             print(gsp_result.iloc[i])


        if gsp_result.iloc[i+1]['Comp'] < gsp_result.iloc[i]['Comp']:
            if format(d['Ton'])!= 'nan': 

                d['Toff'] = gsp_result.iloc[i+1].name
                o = d['Toff']
    #             print(d)
                all_.append(d)
                if (d['Toff'] - d['Ton']) < datetime.timedelta(minutes=30):
                    on = df.index.get_loc(d['Ton'])
                    off = df.index.get_loc(d['Toff'])
                    check_on = 0
                    check_off = 0
                    n = 9
                    if on<9:
                        n = on
                    if ind-off<9:
                        n = ind-off


                    for j in range(1,n):

                        check_on = check_on + gsp_result.iloc[on-j]['Comp']

                        check_off = check_off + gsp_result.iloc[off+j]['Comp']
                    if (check_on == 0) and (check_off == 0):
    #                     print(d)
    #                     print(gsp_result['Power'][d['Ton']:d['Toff']].mean())
                        remove.append(d)
                d = {'Ton':np.nan,'Toff':np.nan}
    #             print(init)

            if format(p['Ton_'])!= 'nan':
                p['Toff_'] = o
    #             print(p)
                powercut.append(p)
    #             print(gsp_result['Power'][p['Ton_']:p['Toff_']].mean())
                p = {'Ton_':np.nan,'Toff_':np.nan}


    # change the dataframe

    gsp_result['Comp2'] = gsp_result['Comp']
    for f in remove:
        if gsp_result['Comp2'][f['Ton']:f['Toff']].mean()<1000:
            gsp_result['Comp2'][f['Ton']:f['Toff']] = 0


    g = gsp_result.loc[gsp_result['Comp2']==0]
    base = g['Power'].mean()

    for g in powercut:
        avg = gsp_result['Power'][g['Ton_']:g['Toff_']].mean()
        if avg-base<300:
            gsp_result['Comp2'][g['Ton_']:g['Toff_']] = 0

    gsp_result.loc[(gsp_result['Power'] <= 1000) & (gsp_result['Comp2'] == 1)]['Comp2'] = 0


    a = []
    d = {'Ton':np.nan,'Toff':np.nan}

    ind = gsp_result.shape[0]-1
    for i in range(0,ind):
        if gsp_result.iloc[i+1]['Comp2'] > gsp_result.iloc[i]['Comp2']:

            d['Ton'] = gsp_result.iloc[i+1].name
        if gsp_result.iloc[i+1]['Comp2'] < gsp_result.iloc[i]['Comp2']:
            if format(d['Ton'])!= 'nan': 

                d['Toff'] = gsp_result.iloc[i+1].name
                a.append(d)
                d = {'Ton':np.nan,'Toff':np.nan}

    gsp_result['AC Power'] = gsp_result['Comp2']
    # AC On
    gsp_result['AC on']=  gsp_result['Comp2']
    for x in range(len(a)-1):
        diff = a[x+1]['Ton']- a[x]['Toff']
        if diff<=datetime.timedelta(minutes=21):
            gsp_result['AC on'][a[x]['Toff']:a[x+1]['Ton']] = 1

    a = []
    d = {'Ton':np.nan,'Toff':np.nan}

    ind = gsp_result.shape[0]-1
    for i in range(0,ind):
        if gsp_result.iloc[i+1]['AC on'] > gsp_result.iloc[i]['AC on']:

            d['Ton'] = gsp_result.iloc[i+1].name
        if gsp_result.iloc[i+1]['AC on'] < gsp_result.iloc[i]['AC on']:
            if format(d['Ton'])!= 'nan': 

                d['Toff'] = gsp_result.iloc[i+1].name
                a.append(d)
                d = {'Ton':np.nan,'Toff':np.nan}


    for t in a:
        on = gsp_result.index.get_loc(t['Ton'])
        off = gsp_result.index.get_loc(t['Toff'])
        ind1 = gsp_result.iloc[on-2].name
        ind2 = gsp_result.iloc[off+2].name


        avg1 = gsp_result['Power'][ind1:t['Ton']].mean()
        avg2 = gsp_result['Power'][t['Toff']:ind2].mean()

        # higher oscillations
        if avg1 >(base+200) or avg2>(base+200):
            avg = base
        else:
            if avg1!=0 and avg2!=0:
                avg = (avg1+avg2)/2
            else:
                if avg1==0 or avg2==0:
                    l = avg1+avg2
                    avg = l
            if avg==0:
                avg = base



    #     print(gsp_result['Power'][t['Ton']:t['Toff']].mean())
        s = gsp_result['Power'][t['Ton']:t['Toff']].mean()-avg
        gsp_result['AC Power'][t['Ton']:t['Toff']] = s
    gsp_result['AC Power'][gsp_result['AC Power'] < 0] = 0            



#     gsp_result.to_csv(folder+meter+'_gsp.csv')
    print(meter,' Done')
    
    #save the result #TODO